In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define dataset paths
dataset_path = '../../'
train_path = os.path.join(dataset_path, "Dataset/Train")
val_path = os.path.join(dataset_path, "Dataset/Validation")
test_path = os.path.join(dataset_path, "Dataset/Test")

# Set a seed for reproducibility
seed = 42
torch.manual_seed(seed)

In [ ]:
def Transform_Image(image):
    image_tensor = transforms.ToTensor()(image).unsqueeze(0)  # Convert to tensor and add batch dimension
    image_tensor.requires_grad = True
    return image_tensor.squeeze(0)

class Transform_Class:
    def __call__(self, image):
        return Transform_Image(image)

In [ ]:
# Define transforms
transformation_for_train = transforms.Compose([
    transforms.Resize((224, 224)),
    Transform_Class(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Directly normalize the tensor
])

transformation_for_valntest = transforms.Compose([
    transforms.Resize((224, 224)),
    Transform_Class(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Directly normalize the tensor
])

In [ ]:
# Load datasets
print(train_path)
train_dataset = datasets.ImageFolder(root=train_path, transform=transformation_for_train)
val_dataset = datasets.ImageFolder(root=val_path, transform=transformation_for_valntest)
test_dataset = datasets.ImageFolder(root=test_path, transform=transformation_for_valntest)
print(train_dataset)
print(val_dataset)
print(test_dataset)

# DataLoader with batch size
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        # Max pool after conv1
        self.pool1 = nn.MaxPool2d(2, 2)  # 2x2 max pooling
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # Max pool after conv2
        self.pool2 = nn.MaxPool2d(2, 2)  # 2x2 max pooling

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Adjusted based on the size after pooling
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        # Pass input through first convolutional layer
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)  # Apply max pooling after conv1
        
        # Pass output of first conv layer through second convolutional layer
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)  # Apply max pooling after conv2

        # Flatten output of second conv layer
        x = x.view(x.size(0), -1)  # Flatten the tensor
        # Pass flattened output through first fully connected layer
        x = self.fc1(x)
        x = F.relu(x)
        # Pass output of first fully connected layer through second fully connected layer
        x = self.fc2(x)
        return x


In [ ]:
# Function to calculate recall and accuracy
def compute_metrics(outputs, labels):
    # Convert the logits to binary predictions
    predicted = (torch.sigmoid(outputs) > 0.5).float()  # Predictions as 0 or 1
    
    # True positives, false positives, false negatives, true negatives
    tp = torch.sum((predicted == 1) & (labels == 1)).item()  # True positives
    fp = torch.sum((predicted == 1) & (labels == 0)).item()  # False positives
    fn = torch.sum((predicted == 0) & (labels == 1)).item()  # False negatives
    tn = torch.sum((predicted == 0) & (labels == 0)).item()  # True negatives
    
    # Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    # Precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0  # Avoid division by zero
    
    # Recall
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0  # Avoid division by zero
    
    # F1-Score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0  # Avoid division by zero
    
    return accuracy, recall, precision, f1

In [ ]:
def train(model, train_loader, test_loader, epochs=10, lr=0.001):
    # Use Adam optimizer to update model weights
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Use BCEWithLogitsLoss for binary classification
    criterion = nn.BCEWithLogitsLoss()
    
    # Performance curves data
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        # Set model to training mode
        model.train()
        epoch_loss = 0.0
        correct = 0
        total = 0
        
        # Iterate over training data
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()  # Get model output
            
            # Compute loss and backpropagate
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            
            # Compute metrics
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # Binary prediction
            accuracy = (predicted == labels).sum().item()
            correct += accuracy
            total += labels.size(0)
        
        epoch_loss /= len(train_loader)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        print(f'--- Epoch {epoch+1}/{epochs}: Train loss: {epoch_loss:.4f}, Train accuracy: {epoch_acc:.4f}')

        # Save model after each epoch
        model_filename = f"../models/baseCNN_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), model_filename)
        print(f"Model saved as {model_filename}")
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        running_loss = 0.0
        true_positive = 0
        false_positive = 0
        false_negative = 0
        true_negative = 0
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.float().to(device)
                outputs = model(inputs).squeeze()
                labels = labels.view(-1)  # Ensure labels have correct shape
                
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                
                predicted = (torch.sigmoid(outputs) > 0.5).float()
                total += labels.size(0)
                
                true_positive += ((predicted == 1) & (labels == 1)).sum().item()
                false_positive += ((predicted == 1) & (labels == 0)).sum().item()
                false_negative += ((predicted == 0) & (labels == 1)).sum().item()
                true_negative += ((predicted == 0) & (labels == 0)).sum().item()
        
        correct = true_positive + true_negative
        avg_loss = running_loss / len(test_loader)
        avg_accuracy = correct / total
        avg_recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
        avg_precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
        avg_f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
        
        print(f'--- Epoch {epoch+1}/{epochs}: Test loss: {avg_loss:.4f}, Test accuracy: {avg_accuracy:.4f}')
        print(f'--- Test Precision: {avg_precision:.4f}, Test Recall: {avg_recall:.4f}, Test F1: {avg_f1:.4f}')
    
    return train_losses, train_accuracies, test_losses, test_accuracies


In [ ]:
model = MNIST_CNN()
model = model.to(device)
train_losses, train_accuracies, test_losses, test_accuracies = train(model,train_loader,test_loader,epochs = 15,lr = 1e-3)
print(train_losses, train_accuracies, test_losses, test_accuracies)